# Създаване на регресионен модел: подготовка и визуализация на данни

## **Линейна регресия за тикви - Урок 2**
#### Въведение

Сега, когато разполагате с инструментите, необходими за изграждане на модели за машинно обучение с Tidymodels и Tidyverse, сте готови да започнете да задавате въпроси към вашите данни. Докато работите с данни и прилагате решения за машинно обучение, е изключително важно да разберете как да зададете правилния въпрос, за да отключите потенциала на вашия набор от данни.

В този урок ще научите:

- Как да подготвите данните си за изграждане на модел.

- Как да използвате `ggplot2` за визуализация на данни.

Въпросът, на който искате да получите отговор, ще определи какъв тип алгоритми за машинно обучение ще използвате. А качеството на отговора, който ще получите, ще зависи в голяма степен от естеството на вашите данни.

Нека видим това чрез практическо упражнение.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>


<!--![Илюстрация от \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Илюстрация от \@allison_horst-->


## 1. Импортиране на данни за тикви и извикване на Tidyverse

Ще ни трябват следните пакети, за да обработим този урок:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) е [колекция от R пакети](https://www.tidyverse.org/packages), създадена да направи науката за данни по-бърза, лесна и забавна!

Можете да ги инсталирате със следната команда:

`install.packages(c("tidyverse"))`

Скриптът по-долу проверява дали имате необходимите пакети за завършване на този модул и ги инсталира, ако някои липсват.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Сега нека стартираме някои пакети и заредим [данните](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), предоставени за този урок!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Един бърз `glimpse()` веднага показва, че има празни стойности и смесица от низове (`chr`) и числови данни (`dbl`). `Date` е от тип символен низ, а има и странна колона, наречена `Package`, където данните са смесица от `sacks`, `bins` и други стойности. Данните, всъщност, са малко хаотични 😤.

Всъщност, не е много често да получите набор от данни, който е напълно готов за създаване на ML модел директно. Но не се притеснявайте, в този урок ще научите как да подготвите суров набор от данни, използвайки стандартни библиотеки на R 🧑‍🔧. Също така ще научите различни техники за визуализиране на данните.📈📊
<br>

> Припомняне: Операторът за тръбопровод (`%>%`) изпълнява операции в логическа последователност, като предава обект напред към функция или израз. Можете да мислите за оператора за тръбопровод като за "и след това" във вашия код.


## 2. Проверка за липсващи данни

Един от най-често срещаните проблеми, с които се сблъскват специалистите по данни, е непълните или липсващи данни. В R липсващите или неизвестни стойности се представят със специална стойност: `NA` (Not Available).

Как можем да разберем дали дадена таблица с данни съдържа липсващи стойности?  
<br>
- Един директен начин е да използваме базовата R функция `anyNA`, която връща логическите стойности `TRUE` или `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Чудесно, изглежда, че липсват някои данни! Това е добро място за начало.

-   Друг начин би бил да използвате функцията `is.na()`, която показва кои отделни елементи в колоните липсват с логическа стойност `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Добре, свършихме работата, но с толкова голяма таблица като тази би било неефективно и практически невъзможно да се прегледат всички редове и колони индивидуално😴.

-   По-интуитивен начин би бил да се изчисли сумата на липсващите стойности за всяка колона:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Много по-добре! Липсват данни, но може би това няма да е от значение за задачата. Нека видим какво ще покаже по-нататъшният анализ.

> Освен страхотните пакети и функции, R разполага с много добра документация. Например, използвайте `help(colSums)` или `?colSums`, за да научите повече за функцията.


## 3. Dplyr: Граматика на манипулиране на данни

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>


<!--![Илюстрация от \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Илюстрация от \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), пакет от Tidyverse, е граматика за манипулиране на данни, която предоставя последователен набор от глаголи, които ви помагат да решите най-често срещаните предизвикателства при обработката на данни. В този раздел ще разгледаме някои от глаголите на dplyr!
<br>


#### dplyr::select()

`select()` е функция в пакета `dplyr`, която ви помага да изберете колони, които да запазите или изключите.

За да направите вашата таблица с данни по-лесна за работа, премахнете няколко от нейните колони, използвайки `select()`, като запазите само колоните, които са ви необходими.

Например, в това упражнение нашият анализ ще включва колоните `Package`, `Low Price`, `High Price` и `Date`. Нека изберем тези колони.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` е функция в пакета `dplyr`, която ви помага да създавате или модифицирате колони, като запазвате съществуващите колони.

Общата структура на `mutate` е:

`data %>%   mutate(new_column_name = what_it_contains)`

Нека използваме `mutate` върху колоната `Date`, като извършим следните операции:

1. Конвертирайте датите (в момента от тип символ) във формат за месец (това са американски дати, така че форматът е `MM/DD/YYYY`).

2. Извлечете месеца от датите в нова колона.

В R, пакетът [lubridate](https://lubridate.tidyverse.org/) улеснява работата с данни от тип дата и час. Затова нека използваме `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` и да видим как да постигнем горните цели. Можем да премахнем колоната `Date`, тъй като няма да ни е необходима за следващите операции.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ура! 🤩

Следващата стъпка е да създадем нова колона `Price`, която представлява средната цена на тиква. Сега, нека изчислим средната стойност на колоните `Low Price` и `High Price`, за да попълним новата колона Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Дааа!💪

„Но чакай малко!“, ще кажете, след като прегледате целия набор от данни с `View(pumpkins)`, „Тук има нещо странно!“🤔

Ако погледнете колоната `Package`, тиквите се продават в много различни конфигурации. Някои се продават в мерки от `1 1/9 bushel`, други в мерки от `1/2 bushel`, някои на тиква, някои на килограм, а някои в големи кутии с различни ширини.

Нека проверим това:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Невероятно!👏

Тиквите изглежда са много трудни за претегляне по последователен начин, затова нека ги филтрираме, като изберем само тиквите със стринга *bushel* в колоната `Package` и поставим това в новия датафрейм `new_pumpkins`.


#### dplyr::filter() и stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): създава подмножество от данни, съдържащо само **редове**, които отговарят на вашите условия, в този случай тикви със стринга *bushel* в колоната `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): открива наличието или отсъствието на даден шаблон в стринг.

Пакетът [`stringr`](https://github.com/tidyverse/stringr) предоставя лесни функции за често срещани операции със стрингове.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Можете да видите, че сме стеснили избора до около 415 реда данни, съдържащи тикви на едро.🤩  
<br>


#### dplyr::case_when()

**Но чакайте! Има още нещо, което трябва да направите**

Забелязахте ли, че количеството в бушел варира за всеки ред? Трябва да нормализирате цените, така че да показвате цената на бушел, а не на 1 1/9 или 1/2 бушел. Време е за малко математика, за да я стандартизирате.

Ще използваме функцията [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), за да *променим* колоната Price в зависимост от някои условия. `case_when` ви позволява да векторизирате множество изрази `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Сега можем да анализираме цената на единица въз основа на измерването им в бушели. Цялото това изследване на бушели с тикви обаче показва колко `важно` е да `разберем естеството на данните си`!

> ✅ Според [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), теглото на един бушел зависи от вида на продукцията, тъй като това е измерване на обем. "Един бушел домати, например, трябва да тежи 56 паунда... Листата и зелените заемат повече пространство с по-малко тегло, така че един бушел спанак е само 20 паунда." Всичко това е доста сложно! Нека не се занимаваме с преобразуване от бушели в паунди, а вместо това да определим цената на бушел. Цялото това изследване на бушели с тикви обаче показва колко важно е да разберем естеството на данните си!
>
> ✅ Забелязахте ли, че тиквите, продавани на половин бушел, са много скъпи? Можете ли да разберете защо? Подсказка: малките тикви са много по-скъпи от големите, вероятно защото има много повече от тях в един бушел, като се има предвид неизползваното пространство, което заема една голяма куха тиква за пай.


Сега, накрая, заради чистото удоволствие от приключението 💁‍♀️, нека също преместим колоната Month на първа позиция, т.е. `преди` колоната `Package`.

`dplyr::relocate()` се използва за промяна на позициите на колоните.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Браво!👌 Сега разполагате с чист и подреден набор от данни, върху който можете да изградите новия си регресионен модел!  
<br>


## 4. Визуализация на данни с ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Инфографика от Dasani Madipalli</figcaption>


<!--![Инфографика от Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Има една *мъдра* поговорка, която гласи:

> "Простият график е донесъл повече информация на ума на анализатора на данни, отколкото всяко друго устройство." --- Джон Туки

Част от ролята на специалиста по данни е да демонстрира качеството и характера на данните, с които работи. За да направят това, те често създават интересни визуализации, или графики, диаграми и таблици, които показват различни аспекти на данните. По този начин те могат визуално да покажат връзки и пропуски, които иначе биха били трудни за откриване.

Визуализациите също могат да помогнат при определянето на най-подходящата техника за машинно обучение за дадените данни. Например, разпръснат график, който изглежда следва линия, показва, че данните са добър кандидат за упражнение с линейна регресия.

R предлага редица системи за създаване на графики, но [`ggplot2`](https://ggplot2.tidyverse.org/index.html) е една от най-елегантните и най-гъвкавите. `ggplot2` ви позволява да съставяте графики чрез **комбиниране на независими компоненти**.

Нека започнем с прост разпръснат график за колоните Price и Month.

В този случай ще започнем с [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), ще предоставим набор от данни и естетическо картографиране (с [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), след което ще добавим слоеве (като [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) за разпръснати графики.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Полезен ли е този график 🤷? Има ли нещо изненадващо в него?

Не е особено полезен, тъй като единственото, което прави, е да показва вашите данни като разпръснати точки за даден месец.  
<br>


### **Как да го направим полезно?**

За да покажем полезни данни в диаграмите, обикновено трябва да групираме данните по някакъв начин. Например, в нашия случай, намирането на средната цена на тиквите за всеки месец би предоставило повече информация за основните модели в нашите данни. Това ни води до още един бърз поглед към **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Групирана агрегация в R може лесно да се изчисли с помощта на

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` променя единицата на анализ от целия набор от данни към отделни групи, например по месеци.

-   `dplyr::summarize()` създава нова таблица с една колона за всяка променлива на групиране и една колона за всяка от статистиките, които сте посочили.

Например, можем да използваме `dplyr::group_by() %>% summarize()`, за да групираме тиквите в групи въз основа на колоната **Month** и след това да намерим **средната цена** за всеки месец.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Кратко!✨

Категориални характеристики, като например месеци, се представят по-добре с помощта на стълбовидна диаграма 📊. Слоевете, отговорни за стълбовидните диаграми, са `geom_bar()` и `geom_col()`. Консултирайте се с `?geom_bar`, за да научите повече.

Да създадем една!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Това е по-полезна визуализация на данни! Изглежда, че най-високата цена за тикви се наблюдава през септември и октомври. Това отговаря ли на вашите очаквания? Защо или защо не?

Поздравления за завършването на втория урок 👏! Подготвихте данните си за изграждане на модел, след което открихте повече прозрения чрез визуализации!



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
